# https://www.iata.org/about/members/Pages/airline-list.aspx?All=true

In [1]:
# Do all imports and installs here
from pyspark.sql.functions import udf
from pyspark.sql.session import SparkSession
import datetime
import pandas as pd
import pyspark.sql.functions as F
import psycopg2

In [2]:
spark = SparkSession.builder\
                    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                    .appName("Project: i94")\
                    .getOrCreate()

In [3]:
# Read tsv file(s)
tsv_i94airline = pd.read_csv('D:/Capstone-Project/Project-Workspace/inputs/Current_Airline_Members.tsv', sep=r'\t', engine='python')

tsv_i94airline.head()


,airline_name,airline,iata_code,icao_designator,country_territory
0,Adria Airways,JP,165,ADR,Slovenia
1,Aegean Airlines,A3,390,AEE,Greece
2,Aer Lingus,EI,053,EIN,Ireland
3,Aero Republica,P5,845,RPB,Colombia
4,Aeroflot,SU,555,AFL,Russian Federation


In [4]:
tsv_i94airline = tsv_i94airline.dropna()

In [5]:
tsv_i94airline.head()

,airline_name,airline,iata_code,icao_designator,country_territory
0,Adria Airways,JP,165,ADR,Slovenia
1,Aegean Airlines,A3,390,AEE,Greece
2,Aer Lingus,EI,053,EIN,Ireland
3,Aero Republica,P5,845,RPB,Colombia
4,Aeroflot,SU,555,AFL,Russian Federation


In [6]:
sp_i94airline = spark.createDataFrame(tsv_i94airline)

In [7]:
sp_i94airline.printSchema()

root
 |-- airline_name: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- icao_designator: string (nullable = true)
 |-- country_territory: string (nullable = true)



In [8]:
sp_i94airline.coalesce(1).write.format('json').mode('overwrite').save('D:/capstone/solarhenge/dim_airline')


In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=i94 user=i94user password=Passw0rd")
cur = conn.cursor()

delete_from_dim_airline = ("""
delete from dim_airline;
""")

insert_into_dim_airline = ("""
insert into dim_airline
(airline_name
,airline
,"3_digit_code"
,icao_designator
,country_territory
) values 
(%s
,%s
,%s
,%s
,%s
);
""")

# a function to insert records
def insert_dim_airline_records(cur, airline_df):
    for i, row in airline_df.iterrows():
        #print('i =',i)
        #print('row =',row)
        cur.execute(insert_into_dim_airline,row)
        
# delete records
cur.execute(delete_from_dim_airline)

# insert records
insert_dim_airline_records(cur, tsv_i94airline)

conn.commit()
conn.close()
